In [1]:
# %load_ext autoreload
# %autoreload 2
# turn off autoreload so that we can use the old model 
# when editing the current project

from coeditor.common import *
import os

os.chdir(proj_root())

In [2]:
from coeditor.retrieval_model import RetrievalEditorModel
from coeditor.api import EditPredictionService, BasicQueryEditEncoder, BatchArgs

model_path = get_model_dir(True) / "coeditor-large-query-basic"
model = RetrievalEditorModel.load(model_path)
model.to("cuda:1")
batch_args = BatchArgs.service_default()
service = EditPredictionService(
    proj_root(),
    model,
    batch_args=batch_args,
    encoder=BasicQueryEditEncoder(
        max_ref_tks=batch_args.max_ref_tks,
        max_query_tks=batch_args.max_query_tks,
        max_output_tks=batch_args.max_output_tks,
    ),
)


In [3]:
service.suggest_edit(Path("src/coeditor/history.py"), 136)
# service.suggest_edit(Path("src/coeditor/api.py"), 277)
# service.suggest_edit(Path("src/spot/utils.py"), (218, 0))


stub files: dict_keys(['coeditor.api', 'coeditor.common', 'coeditor.retrieval_model', 'coeditor.history'])
num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
========== Predicted code change ==========
* Modified: 
        # path: coeditor.history/empty_module
        def empty_module(mname: ModuleName) -> PythonModule:
    -       return PythonModule.from_cst(cst.Module([]), mname, False)
    +       return PythonModule.from_cst(cst.Module([]), mname, None)


In [4]:
service.tlogger.as_dataframe()


,name,count,avg_time,total_time
1,construct project edit,1,2.321746,2.321746
3,run model,1,0.820002,0.820002
2,encode edits,1,0.561629,0.561629
0,get target element,1,0.424238,0.424238
